# Using Naive Bayes to classify the articles:

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion

import pandas as pd
import numpy as np
import glob
import re
import string
import datetime

In [2]:
category = {
    1: 'politik', 2: 'wirtschaft', 3: 'finanzen', 4: 'feuilleton', 5: 'sport', 6: 'gesellschaft', 7: 'stil', 
    8: 'technik-motor', 9: 'wissen', 10: 'reise', 11: 'beruf-chance'
}

In [3]:
def train_tables(category):
    raw_train = []
    if category is 'aktuell':
        path = f"../new_data/aktuell/"
    else:
        path = f"../new_data/{category}/"
        
    all_files = glob.glob(path + '*.csv')

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        raw_train.append(df)
    faz_articles = pd.concat(raw_train, axis=0, ignore_index=True)
    faz_articles['label'] = category

    return faz_articles

def test_tables(category):
    raw_test = []
    if category is 'aktuell':
        path = f"../data/aktuell/"
    else:
        path = f"../data/{category}/"
        
    all_files = glob.glob(path + '*.csv')

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        raw_test.append(df)
    faz_article = pd.concat(raw_test, axis=0, ignore_index=True)
    faz_article['label'] = category

    return faz_article

---
---
# Generate Test and Train Data

**This for-loop goes through the data frames built in the first step and then collects the words from each entry in a list.**

## Train Data ['detailed']:

In [4]:
frames = []

for key, value in category.items():
    raw_faz = train_tables(value)
    frames.append(raw_faz)
    faz_train = pd.concat(frames, axis=0, ignore_index=True)

In [5]:
faz_train = faz_train.drop_duplicates()

In [6]:
faz_pol = faz_train[faz_train.label == 'politik']
faz_pol['goal_val'] = 1

faz_sport = faz_train[faz_train.label == 'sport']
faz_sport['goal_val'] = 2

faz_eco = faz_train[faz_train.label == 'wirtschaft']
faz_eco['goal_val'] = 3


/anaconda3/envs/toolbox/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/toolbox/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/envs/toolbox/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [7]:
faz_train_2 = faz_train.copy()

faz_train_2 = faz_train_2[faz_train_2.label != 'politik']
faz_train_2 = faz_train_2[faz_train_2.label != 'sport']
faz_train_2 = faz_train_2[faz_train_2.label != 'wirtschaft']
faz_rem = faz_train_2

faz_rem['goal_val'] = 0

In [8]:
train_conc = [faz_pol,faz_sport,faz_eco,faz_rem]
train = pd.concat(train_conc)
train.head()

,link,published,title,detailed,label,goal_val
0,https://www.faz.net/aktuell/politik/ausland/na...,"Mon, 08 Apr 2019 20:52:28 +0200",Nach Heimatschutzministerin geht in Amerika au...,Randolph Alles auf einer Pressekonferenz am 26...,politik,1
1,https://www.faz.net/aktuell/politik/trumps-pra...,"Mon, 08 Apr 2019 20:59:02 +0200",Freiwillig oder nicht? Ministerin verlässt Tru...,Nicht immer einer Meinung: Nielsen und Trump i...,politik,1
2,https://www.faz.net/aktuell/politik/ausland/eu...,"Mon, 08 Apr 2019 19:57:02 +0200",Europäische Rechtspopulisten planen neue Frakt...,"Nationalistische Internationale: Kotro, Meuthe...",politik,1
3,https://www.faz.net/aktuell/brexit/brexit-aufs...,"Mon, 08 Apr 2019 20:30:09 +0200",Abstimmung zum Brexit-Aufschub: Londoner Börse...,Brexit-Gegner demonstrieren am Montag in Londo...,politik,1
4,https://www.faz.net/aktuell/politik/ausland/di...,"Mon, 08 Apr 2019 19:04:29 +0200",Die Ergebnisse von Macrons Bürgerdialog,Frankreichs Premierminister Edouard Philippe a...,politik,1


In [9]:
#train.title[0]

In [10]:
train_detailed = train.drop(columns=['link', 'published','title','label'])
train_detailed.head()

,detailed,goal_val
0,Randolph Alles auf einer Pressekonferenz am 26...,1
1,Nicht immer einer Meinung: Nielsen und Trump i...,1
2,"Nationalistische Internationale: Kotro, Meuthe...",1
3,Brexit-Gegner demonstrieren am Montag in Londo...,1
4,Frankreichs Premierminister Edouard Philippe a...,1


## Train Data ['title']:

In [11]:
train_titles = train.drop(columns=['link', 'published','detailed','label'])
train_titles.tail()

,title,goal_val
18125,Gegen den Trend: Mehr Frauen machen sich selbs...,0
18369,Der Handwerk-Gehaltsatlas: Gefragt – aber nich...,0
18379,Alibaba-Gründer Jack Ma: „Die 72 Stunden-Woche...,0
18669,Karliczek stellt Bericht vor: So steht es um d...,0
18859,Kuriose Stellenanzeigen: Wenn Arbeitgeber über...,0


In [12]:
train_titles = train_titles.rename(index=str, columns={"title": "detailed"})

In [13]:
combined = [train_detailed,train_titles]
train_tot = pd.concat(combined)

In [14]:
X_train = train['detailed']
X_train_title = train_titles['detailed']
X_train_tot = train_tot['detailed']

In [15]:
y_train = train['goal_val']
y_train_title = train_titles['goal_val']
y_train_tot = train_tot['goal_val']

## Test Data['detailed']:

In [16]:
frames = []

for key, value in category.items():
    raw_faz = test_tables(value)
    frames.append(raw_faz)
    faz_test = pd.concat(frames, axis=0, ignore_index=True)

In [17]:
faz_test = faz_test.drop_duplicates()

In [18]:
faz_pol_t = faz_test[faz_test.label == 'politik']
faz_pol_t['goal_val'] = 1

faz_sport_t = faz_test[faz_test.label == 'sport']
faz_sport_t['goal_val'] = 2

faz_eco_t = faz_test[faz_test.label == 'wirtschaft']
faz_eco_t['goal_val'] = 3

/anaconda3/envs/toolbox/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/toolbox/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/envs/toolbox/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [19]:
faz_test_2 = faz_test.copy()

faz_test_2 = faz_test_2[faz_test_2.label != 'politik']
faz_test_2 = faz_test_2[faz_test_2.label != 'sport']
faz_test_2 = faz_test_2[faz_test_2.label != 'wirtschaft']
faz_rem_t = faz_test_2

faz_rem_t['goal_val'] = 0

In [20]:
test_conc = [faz_pol_t,faz_sport_t,faz_eco_t,faz_rem_t]
test = pd.concat(test_conc) 

In [21]:
test_detailed = test.drop(columns=['link', 'published','title','label'])

## Test Data ['title']:

In [22]:
test_titles = test.drop(columns=['link', 'published','detailed','label'])
test_titles = test_titles.rename(index=str, columns={"title": "detailed"})

In [23]:
comb = [test_detailed,test_titles]
test_tot = pd.concat(comb)

In [24]:
X_test = test['detailed']
X_test_title = test_titles['detailed']
X_test_tot = test_tot['detailed']

In [25]:
y_test = test['goal_val']
y_test_title = test_titles['goal_val']
y_test_tot = test_tot['goal_val']

In [26]:
print("# of entries - Training dataset: ", X_train.shape[0])
print("# of entries - Test dataset: ", X_test.shape[0])
print('\n')
print("# of entries - Title Training dataset: ", X_train_title.shape[0])
print("# of entries - Title Test dataset: ", X_test_title.shape[0])

# of entries - Training dataset:  1446
# of entries - Test dataset:  480


# of entries - Title Training dataset:  1446
# of entries - Title Test dataset:  480


In [27]:
print('Trainset:')
print(train_tot.goal_val.value_counts())
print('\n')
print('Testset:')
print(test_tot.goal_val.value_counts())

Trainset:
0    1244
1     650
2     580
3     418
Name: goal_val, dtype: int64


Testset:
0    466
1    196
2    164
3    134
Name: goal_val, dtype: int64


---
### Apply Bayes Algorithm:
---

In [28]:
stopword = pd.read_csv('/Users/torben/PycharmProjects/toolbox/stopwords/stopwords.csv', index_col=None, header=0)
stopword_u = pd.read_csv('/Users/torben/PycharmProjects/toolbox/stopwords/stopwordsupper.csv', index_col=None, header=0)
swl = [stopword,stopword_u]

In [29]:
stopword_list = pd.concat(swl)
stopwords = stopword_list['words'].values.tolist()

---

In [30]:
#count_vector = CountVectorizer(stop_words=stopwords,
#                               analyzer='word',
#                               ngram_range=(1,4)
#                              )
#
#training_data = count_vector.fit_transform(X_train_tot)
#testing_data = count_vector.transform(X_test_tot)

In [31]:
tf_vector = TfidfVectorizer(stop_words=stopwords,
                               analyzer='word',
                               ngram_range=(1,2)
                              )

training_data = tf_vector.fit_transform(X_train_tot)
testing_data = tf_vector.transform(X_test_tot)

/anaconda3/envs/toolbox/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aussen', 'ausserhalb', 'dat', 'dreissig', 'einigermassen', 'gewissermassen', 'inf', 'müsst', 'regelmässig', 'schliesslich', 'weiss', 'zb'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [32]:
#vecA = CountVectorizer(stop_words=stopwords,
#                               analyzer='word',
#                               ngram_range=(1,4))
#vecA.fit_transform(X_train)
#
#
#vecB = TfidfVectorizer(stop_words=stopwords,
#                               analyzer='word',
#                               ngram_range=(1,2))
#vecB.fit_transform(X_train_title)

In [33]:
#combined_features = FeatureUnion([('CountVectorizer', vecA),('TfidfVectorizer', vecB)])

In [34]:
#combined_features.transform(X_test_tot)

In [35]:
print('Trainset:')
print(training_data.shape)
print('\n')
print('Testset:')
print(testing_data.shape)

Trainset:
(2892, 33664)


Testset:
(960, 33664)


In [36]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train_tot)

predictions = naive_bayes.predict(testing_data)

print(f'Accuracy score: {accuracy_score(y_test_tot, predictions)*100} %')
#print("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Accuracy score: 73.95833333333334 %


---
## Using the StochasticGradientDescent with new feature vector:

In [50]:
sgd = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(training_data, y_train_tot)  

prediction = sgd.predict(testing_data)

test_accuracy = accuracy_score(y_test_tot, prediction)

print(f'The TEST prediction is {test_accuracy*100} % ACCURATE')

The TEST prediction is 78.54166666666667 % ACCURATE
